#### Packages to import

In [1]:
#! pip install dateparser
#!pip install pyspellchecker
from dateparser import parse
import spacy
from datetime import datetime
from spellchecker import SpellChecker
import autocomplete
import requests, json 
nlp=spacy.load('en_core_web_sm')

C:\Users\imtkhan\AppData\Roaming\Python\Python37\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.1). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


#### a.Time/Date elements

In [2]:
def gettime_component(query,out={}):
    doc=nlp(query)
    for chunk in doc.noun_chunks:
        phrase=parse((chunk.text))
        if phrase:
            out[str(chunk.text)]=[{'gte': str(datetime.now() if phrase>datetime.now() else phrase) ,
                         'lte': str(datetime.now() if phrase<datetime.now() else phrase)}]
    return out  

#test
gettime_component('Profit of my organization in last month')

{'last month': [{'gte': '2020-07-14 18:36:58.921890',
   'lte': '2020-08-14 18:36:58.922894'}]}

#### b. NER

In [3]:
def getlocations(query,out={}):
    doc=nlp(query)
    for entity in doc.ents:
        if entity.label_=='GPE':
            out[entity.text]='LOCATION'
    return out

#test
getlocations('Profit of my organization in last month in US and India')

{'US': 'LOCATION', 'India': 'LOCATION'}

#### Association between the tokens in the user query

In [4]:
def getassociations(query,out={}):
    doc = nlp(query)
    for token in doc:
        out[str(token.text)]={'dep':token.dep_,'head':token.head.text,
                              'headtag':token.head.pos_,
                              'children':[child for child in token.children]}
    return out


# Text: The original token text.
# Dep: The syntactic relation connecting child to head.
# Head text: The original text of the token head.
# Head POS: The part-of-speech tag of the token head.
# Children: The immediate syntactic dependents of the token.

###test
getassociations("Profit of my organization in last month in US and India")

{'Profit': {'dep': 'ROOT',
  'head': 'Profit',
  'headtag': 'NOUN',
  'children': [of, in, in]},
 'of': {'dep': 'prep',
  'head': 'Profit',
  'headtag': 'NOUN',
  'children': [organization]},
 'my': {'dep': 'poss',
  'head': 'organization',
  'headtag': 'NOUN',
  'children': []},
 'organization': {'dep': 'pobj',
  'head': 'of',
  'headtag': 'ADP',
  'children': [my]},
 'in': {'dep': 'prep', 'head': 'Profit', 'headtag': 'NOUN', 'children': [US]},
 'last': {'dep': 'amod', 'head': 'month', 'headtag': 'NOUN', 'children': []},
 'month': {'dep': 'pobj', 'head': 'in', 'headtag': 'ADP', 'children': [last]},
 'US': {'dep': 'pobj',
  'head': 'in',
  'headtag': 'ADP',
  'children': [and, India]},
 'and': {'dep': 'cc', 'head': 'US', 'headtag': 'PROPN', 'children': []},
 'India': {'dep': 'conj', 'head': 'US', 'headtag': 'PROPN', 'children': []}}

#### Identification and correction of spell errors in the user query. 

In [5]:
def correctSpellings(query):
    spell = SpellChecker()
    return ' '.join(spell.correction(word) for word in query.split())

##test
correctSpellings("Proft of my organizatn in last mnth in US and India")

'profit of my organization in last month in US and India'

#### Next phrase prediction while the user types a query

In [6]:
def getSuggestions(term):
    URL="http://suggestqueries.google.com/complete/search?client=chrome&q="+term
    apiurl = URL
    r = requests.get(apiurl)
    text=json.loads(r.content.decode('utf-8'))
    return text[1]

### test
getSuggestions('last 12')

['last 12 presidents',
 'last 12 months',
 'last 12 hour radar',
 'last 12 months from today',
 'last 12 months sql',
 'last 12 hour radar loop',
 'last 12 verses of mark',
 'last 12 hours weather radar']

#### Brand name or product name identification from an NLP query without any lookup table.

In [7]:
def getBrandNames(query,out={}):
    doc=nlp(query)
    for d in doc.ents:
        if d.label_=='ORG':
            out[d.text]=d.label_
    return out

#### test
getBrandNames('profit of Nike in last month in US and India')    

{'Nike': 'ORG'}